<a href="https://colab.research.google.com/github/eswens13/deep_learning/blob/master/keras/mnist_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST Classifier

Exploring an introductory deep learning problem to solidify some of the concepts I've learned thusfar.

## Prepare Environment and Data

We need to pull in the MNIST dataset and set up the environment so that we can checkpoint the model.  I have decided to use a utility called ngrok that acts as a tunnel between my local machine and the Colab server so that I can run TensorBoard and examine the training process.  This first cell spits out a URL where the TensorBoard visualizations can be viewed.

Note that the samples in the MNIST dataset are grayscale so we have to artificially append a channel dimension to them.

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
LINUX_BASE_PATH = '/content/drive/My\ Drive/deep_learning'
PYTHON_BASE_PATH = '/content/drive/My Drive/deep_learning'
LINUX_MNIST_DIR = os.path.join(LINUX_BASE_PATH, 'mnist')
PYTHON_MNIST_DIR = os.path.join(PYTHON_BASE_PATH, 'mnist')
os.system('mkdir -p ' + LINUX_MNIST_DIR)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


0

In [2]:
from keras.datasets import mnist
from keras.utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# One-hot encode the labels.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Need to give a "channels" dimension even though the images are grayscale.
X_train = X_train[:,:,:,None]
X_test = X_test[:,:,:,None]

print("Data Shapes:\n\tX_train: {}\n\ty_train: {}\n\tX_test: {}\n\ty_test{}"\
      .format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

import os

# Get a tool called ngrok to use as a tunnel between my local machine and the
# Google Colab server. This will allow us to use TensorBoard to visualize
# helpful metrics of the network.
#
# Tutorial:
#   https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
#
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Now the ngrok exectuable is extracted to the current directory. Check to make
# sure there is a log directory for Keras to use.
cwd = os.getcwd()
LOG_DIR = os.path.join(cwd, 'log')
print("Log Dir: {}".format(LOG_DIR))
if not os.path.exists(LOG_DIR):
  os.system('mkdir -p {}'.format(LOG_DIR))

# Run tensorboard in the background.
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# Tell ngrok (in the background) to tunnel TensorBoard port 6006 to the outside
# world.
get_ipython().system_raw('./ngrok http 6006 &')

# Get the URL that I can use to hook into TensorBoard from my local machine.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
Data Shapes:
	X_train: (60000, 28, 28, 1)
	y_train: (60000, 10)
	X_test: (10000, 28, 28, 1)
	y_test(10000, 10)
--2019-05-29 03:26:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.207.111.186, 35.173.3.255, 52.200.123.104, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.207.111.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  38.6MB/s    in 0.4s    

2019-05-29 03:26:16 (38.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Log Dir: /content/log
https://0fe2da97.ngrok.io


## Network Architecture

This next cell defines the network architecture.

In [0]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.regularizers import l2

import numpy as np

# Build the Keras model
def get_keras_model():
  
  model = Sequential();
  num_filters = 32
  kernel_size = [3,3]
  stride_size = [1,1]
  pad_str = 'same'
  format_str = 'channels_last'
  act_str = 'relu'
  
  model.add(Conv2D(num_filters, \
                   kernel_size, \
                   strides=stride_size, \
                   padding=pad_str, \
                   data_format=format_str, \
                   use_bias=True, \
                   activation=act_str))
  
  pool_window = [2,2]
  stride_size = [2,2]
  pad_str = 'valid'
  model.add(MaxPooling2D(pool_size=pool_window, \
                         strides=stride_size, \
                         padding=pad_str, \
                         data_format=format_str))

  num_filters = 64
  model.add(Conv2D(num_filters, \
                   kernel_size, \
                   strides=stride_size, \
                   padding=pad_str, \
                   data_format=format_str, \
                   use_bias=True, \
                   activation=act_str))
  
  model.add(MaxPooling2D(pool_size=pool_window, \
                         strides=stride_size, \
                         padding=pad_str, \
                         data_format=format_str))
  
  model.add(Flatten(data_format=format_str))
  
  model.add(Dense(512, activation=act_str, use_bias=True))
  model.add(Dense(128, activation=act_str, use_bias=True))
  model.add(Dense(64, activation=act_str, use_bias=True))
  
  # Output Layer
  model.add(Dense(10, activation=act_str, use_bias=True))
  
  # Compile the model.
  adam = Adam(lr=1e-5)
  model.compile(loss='categorical_crossentropy', \
                optimizer=adam, \
                metrics=['accuracy'])
  
  return model

## Train the Model

In [4]:
# Create a Keras callback so that it outputs to TensorBoard rather than this
# console.
from keras.callbacks import TensorBoard
from datetime import datetime

tbCallback = TensorBoard(log_dir=LOG_DIR, \
                         histogram_freq=1, \
                         write_graph=False, \
                         write_grads=True, \
                         batch_size=100, \
                         write_images=False)

now = datetime.now()
curr_date_time = now.strftime("%m%d%Y_%H%M%S")
model_file = "{}.h5".format(curr_date_time)
full_model_path = os.path.join(PYTHON_MNIST_DIR, model_file)

# Run training on the model.
model = get_keras_model()
model.fit(X_train, y_train, \
          epochs=5, batch_size=100, verbose=2, callbacks=[tbCallback], \
          validation_data=(X_test, y_test))
model.save(full_model_path)
del model

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
 - 7s - loss: 5.0778 - acc: 0.1777 - val_loss: 4.5308 - val_acc: 0.4061
Epoch 2/5
 - 3s - loss: 4.4034 - acc: 0.4811 - val_loss: 4.1595 - val_acc: 0.5709
Epoch 3/5
 - 3s - loss: 4.1531 - acc: 0.5895 - val_loss: 3.9850 - val_acc: 0.6306
Epoch 4/5
 - 3s - loss: 4.0481 - acc: 0.6269 - val_loss: 3.8886 - val_acc: 0.6567
Epoch 5/5
 - 3s - loss: 3.9752 - acc: 0.6641 - val_loss: 3.8359 - val_acc: 0.6838


In [6]:
from keras.models import load_model

model_to_eval = load_model(full_model_path)
scores = model_to_eval.evaluate(X_test, y_test)
for i in range(len(model_to_eval.metrics_names)):
  print("{}: {}".format(model_to_eval.metrics_names[i], scores[i]))

10000/10000 [==============================] - 1s 69us/step
loss: 3.835926488494873
acc: 0.6838
